In [4]:
from goprocam import GoProCamera, constants
from threading import Thread
import cv2
import socket
import math
import imutils
import numpy as np
from time import time
from random import randint
#number of tracker points, depends on how many you want, once you choose the value, you need to obey
numTracker = 1
#define lower and upper boundries for black color
lower_bound = np.array([0,0,0])
upper_bound = np.array([96,96,96])#before it's 96,96,96
#setup the tracker, read more https://ehsangazar.com/object-tracking-with-opencv-fd18ccdd7369
tracker = cv2.MultiTracker_create()
#stream the video from webcam
cap=cv2.VideoCapture(0)
ret,frame = cap.read()
center = []
points = []
csv_file = []
frame_index = 0
#create a black image and an array to contain three points, draw red x and y axis on the picture
img = np.zeros((512,512,3), np.uint8)
#the array of select boxes
bboxes = []
#the color of the select boxes
colors = []
while True:
#select a bounding box to do object tracking
    bbox = cv2.selectROI(frame,False,False)
    bboxes.append(bbox)
    colors.append((randint(0,255),randint(0,255),randint(0,255)))
    if cv2.waitKey(0) & 0xFF == 27:
        break
# Initialize tracker with first frame and bounding box
#ok = tracker.init(frame, bbox)
for bbox in bboxes:
    tracker.add(cv2.TrackerKCF_create(),frame,bbox)
while True:
    ret, frame = cap.read()
    cv2.imshow("webcam", frame)
    success, boxes = tracker.update(frame)
    if success:
        for i, newbox in enumerate(boxes):
         # Tracking success
            p1 = (int(newbox[0]), int(newbox[1]))
            p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
            cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
            if len(center) == numTracker:
                #np.savetxt("web_cam.csv", center, delimiter=",", fmt='%d', header= "Frame Number, X, Y", comments = '')
                csv_file = csv_file + center
                frame_index = frame_index + 1
                center = []
            center.append([frame_index, int(newbox[0] + newbox[2]/2), int(newbox[1] + newbox[3]/2)])
            if len(points) == numTracker:
                points = []
            points.append((int(newbox[0] + newbox[2]/2), int(newbox[1] + newbox[3]/2)))
        #draw center
            cv2.circle(mask,(int(newbox[0] + newbox[2]/2),int(newbox[1] + newbox[3]/2)),2,(0,0,255),3)
    else :
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
            #blurred = cv2.GaussianBlur(frame,(11,11),0)
            #hsv = cv2.cvtColor(blurred,cv2.COLOR_BGR2HSV)
            #construct a mask of black
            #mask = cv2.inRange(hsv, lower_bound, upper_bound)
            #mask = cv2.erode(mask, None, iterations = 2)
            #mask = cv2.dilate(mask, None, iterations = 2)
            #cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            #cnts = imutils.grab_contours(cnts)
            #for cn in cnts:
                #c = max(cn, key=cv2.contourArea)
                #((x,y),radius) = cv2.minEnclosingCircle(cn)
                #if radius > 15:#this may change due to the size of mark
                    #cv2.circle(frame,(int(x),int(y)),int(radius),(255,255,255),-1)
                    #points.append(int(x),int(y))
                    #csv_file.append([frame_index, int(x), int(y)])
    for point in range(len(points)-1):
        cv2.line(img,points[point],points[point+1],(255,0,0),5)
    cv2.imshow("plot",img)
    cv2.imshow("tracking",frame)
    np.savetxt("web_cam.csv", csv_file, delimiter=",", fmt='%d', header= "Frame Number, X, Y", comments = '')
    #reload the img pic
    img = np.zeros((512,512,3), np.uint8)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [2]:
import cv2
import socket
import math
import imutils
import numpy as np
from time import time
from matplotlib import pyplot as plt
from matplotlib import image as mpimg

def arrangeArray(a):
    if(len(a)<5):
        return a
    first = 0
    second = 0
    firstLargestX = ()
    secondLargestX = ()
        #a = [(482,448),(397,412),(314,378),(247,340)]
    if a[0][0] < 300 and a[2][0] < 410 and a[3][0] < 500 and a[4][0] < 500:
        for x in a:
            if x[0] > first:
                second = first
                first = x[0]
                secondLargestX = firstLargestX
                firstLargestX = x
            elif x[0] > second and x[0] != first:
                second = x[0]
                secondLargestX = x
        #print(firstLargestX)
        #print(secondLargestX)
        a.remove(firstLargestX)
        a.remove(secondLargestX)
        if(firstLargestX[1] <= secondLargestX[1]):
            a.append(secondLargestX)
            a.append(firstLargestX)
        else:
            a.append(firstLargestX)
            a.append(secondLargestX)
        #print(a)
        if a[1][1] < a[2][1]:
            b = a[1]
            c = a[2]
            a.remove(a[1])
            #print("after first remove", a)
            a.remove(a[1])
            #print("after second remove", a)
            a.append(b)
            a.append(c)
        else:
            b = a[2]
            c = a[1]
            a.remove(a[1])
            a.remove(a[1])
            a.append(b)
            a.append(c)
    #print(a)
    return a


WRITE = False
center1 = [] #has to differ than the previous cut
frame_index = 0
#define lower and upper boundries for black color
lower_bound = np.array([0,0,0])
upper_bound = np.array([96,96,96])#before it's 96,96,96
#check if save button is pressed
s_pressed = 0
#stream the video from webcam
cap=cv2.VideoCapture(0)
frames_per_second = 20.0
out = None
#create a black image and an array to contain three points, draw red x and y axis on the picture
img = np.zeros((512,512,3), np.uint8)
#draw x axis line
cv2.line(img,(0,0),(512,0),(0,0,255),2)
for x in range(0,51):
    cv2.line(img,(10*x,0),(10*x,5),(0,0,255),2)
#draw y axis line
cv2.line(img,(0,0),(0,512),(0,0,255),2)
for y in range(0,51):
    cv2.line(img,(0,10*x),(5,10*x),(0,0,255),2)
points = []
while True:
    ret,frame = cap.read()
    #get the frame size
    if ret:
        frame_dimension = frame.shape
        frame_height = frame_dimension[0]
        frame_width = frame_dimension[1]
        #define the codec and create a video_writer object
        if out is None:
                fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Be sure to use the lower case
#                 out  = cv2.VideoWriter('center_points_{0}.mp4',fingerVid9)
                out = cv2.VideoWriter('test_web_cam.mp4',
                             fourcc, frames_per_second, (frame_width, frame_height), True)
    
    cv2.imshow("webcam",frame)
    #blur the picture and transform it to hsv
    blurred = cv2.GaussianBlur(frame,(11,11),0)
    hsv = cv2.cvtColor(blurred,cv2.COLOR_BGR2HSV)
    #construct a mask of black
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    mask = cv2.erode(mask, None, iterations = 2)
    mask = cv2.dilate(mask, None, iterations = 2)
    #find contours in the mask
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnts = imutils.grab_contours(cnts)
    for cn in cnts:
        c = max(cn, key=cv2.contourArea)
        ((x,y),radius) = cv2.minEnclosingCircle(cn)
        if radius > 15:#this may change due to the size of mark
            cv2.circle(frame,(int(x),int(y)),int(radius),(255,255,255),-1)
            #fill up a point array
            points.append((int(x),int(y)))
            #fill up the array for csv file
            if cv2.waitKey(1) & 0xFF == ord('s'):
                s_pressed = 1
            if s_pressed == 1:
                center1.append([frame_index, int(x), int(y)])
            #print("frame is",frame_index,"x is", int(x), "y is", int(y))
    frame_index = frame_index + 1
    #points = arrangeArray(points)
    for size in range(len(points)-1):
        cv2.line(img,points[size],points[size+1],(255,0,0),5)
         #cv2.drawContours(frame,[c],-1,(255,255,255),2)
    #refresh the points
    points = []
    out.write(frame)
    cv2.imshow("plot",img)
    #renew the image
    if s_pressed == 1:
        img = np.zeros((512,512,3), np.uint8)
        cv2.putText(img, 'Saved!', (400,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
        cv2.line(img,(0,0),(512,0),(0,0,255),2)
        for x in range(0,51):
            cv2.line(img,(10*x,0),(10*x,5),(0,0,255),2)
        cv2.line(img,(0,0),(0,512),(0,0,255),2)
        for y in range(0,51):
            cv2.line(img,(0,10*y),(5,10*y),(0,0,255),2)
    else:
        img = np.zeros((512,512,3), np.uint8)
        #cv2.putText(img, 'Saved!',(500,10),1,(0,0,0),2)
        cv2.line(img,(0,0),(512,0),(0,0,255),2)
        for x in range(0,51):
            cv2.line(img,(10*x,0),(10*x,5),(0,0,255),2)
        cv2.line(img,(0,0),(0,512),(0,0,255),2)
        for y in range(0,51):
            cv2.line(img,(0,10*y),(5,10*y),(0,0,255),2)
    cv2.imshow("detect",frame)
    #write the modified video output file
    np.savetxt("web_cam1.csv", center1, delimiter=",", fmt='%d', header= "Frame Number, X, Y", comments = '')#the csv file name and array should be matched
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

In [12]:
    def arrangeArray(a):    
        first = 0
        second = 0
        firstLargestX = ()
        secondLargestX = ()
        #a = [(482,448),(397,412),(314,378),(247,340)]
        if a[0][0] < 300 and a[3][0] < 500 and a[4][0] < 500 and a[2][0] < 420:
            for x in a:
                if x[0] > first:
                    second = first
                    first = x[0]
                    secondLargestX = firstLargestX
                    firstLargestX = x
                elif x[0] > second and x[0] != first:
                    second = x[0]
                    secondLargestX = x
            print(firstLargestX)
            print(secondLargestX)
            a.remove(firstLargestX)
            a.remove(secondLargestX)
            if(firstLargestX[1] <= secondLargestX[1]):
                a.append(secondLargestX)
                a.append(firstLargestX)
            else:
                a.append(firstLargestX)
                a.append(secondLargestX)
            print(a)
            if a[1][1] < a[2][1]:
                b = a[1]
                c = a[2]
                a.remove(a[1])
                print("after first remove", a)
                a.remove(a[1])
                print("after second remove", a)
                a.append(b)
                a.append(c)
            else:
                b = a[2]
                c = a[1]
                a.remove(a[1])
                a.remove(a[1])
                a.append(b)
                a.append(c)
        print(a)
        return a
    a = [(482,448),(397,412),(314,378),(247,340)]
    b = arrangeArray(a)
    print("b is", b)

[(482, 448), (397, 412), (314, 378), (247, 340)]
b is [(482, 448), (397, 412), (314, 378), (247, 340)]
